In [68]:
import rasterio
from rasterio.features import geometry_mask
import geopandas as gpd
from shapely.geometry import box
import geopandas as gpd
import fiona.crs as crs

import os
import glob
import send2trash

In [69]:
def clip_shapefile(sar_image_path, output_file_path, input_file_path):

    with rasterio.open(sar_image_path) as src:
        left, bottom, right, top = src.bounds

    # Specify the clipping extent in EPSG:4326 (WGS 84)
    clip_extent = gpd.GeoDataFrame(geometry=[box(left, bottom, right, top)], crs=crs.CRS.from_epsg(4326))

    input_data = gpd.read_file(input_file_path)

    # Clip the data to the specified extent
    clipped_data = gpd.overlay(input_data, clip_extent, how='intersection')

    # Save the clipped data to a new shapefile
    clipped_data.to_file(output_file_path, driver='ESRI Shapefile')


In [70]:
def create_binary_image(sar_image_path, shapefile_path, output_image_path):
    # Read SAR image
    with rasterio.open(sar_image_path) as src:
        sar_data = src.read(1)
        transform = src.transform
        profile = src.profile

    # Read shapefile
    gdf = gpd.read_file(shapefile_path)
    

    try:
        print(sar_data.shape)
        mask = geometry_mask(gdf.geometry, out_shape=sar_data.shape, transform=transform, invert=True)

        # Create a new image with the same dimensions as the SAR image
        binary_image = mask.astype('uint8') * 255

        # Update the profile for the output image
        profile.update(count=1, dtype='uint8')

        # Write the new image
        with rasterio.open(output_image_path, 'w', **profile) as dst:
            dst.write(binary_image, 1)
    
    except ValueError as ve:
        send2trash.send2trash(sar_image_path)
        # os.remove(sar_image_path)

In [71]:
def generate_water_masks(input_dir):
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')))
    input_shapefile_path = "./Brazil_Shapefile/Areas_merged.shp"
    
    for input_file in input_files:
        
        filename = input_file.replace("./data_new\\", "")
        filename_tif = filename.replace(".tif", "")
        shapefile_path = "./shapefiles/"+filename_tif+".shp"

        # print("Starting "+filename)

        clip_shapefile(input_file, shapefile_path, input_shapefile_path)

        output_file_path = "./output_tif/"+filename_tif+"_new_mask.tif"
        create_binary_image(input_file, shapefile_path, output_file_path)
        print("Done "+filename)

In [75]:
def cleanup(shapefiles_dir):
    
    extensions = ["*.shp", "*.cpg", "*.dbf", "*.shx", "*.prj"]

    for extension in extensions:
        shapefiles = glob.glob(os.path.join(shapefiles_dir, extension))
        # Delete each shapefile
        for shapefile in shapefiles:
            os.remove(shapefile)

        print("All shapefiles deleted successfully.")

In [73]:
generate_water_masks("./data_new/")

(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_0.tif
(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1024.tif
(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1536.tif
(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_2048.tif
(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_2496.tif
(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_512.tif
(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_1024_0.tif
(512, 512)
Done subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_1024_1024.tif
(

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


(512, 512)
Done subset_10_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101122_20240131T101151_052349_065480_4193_0_1536.tif


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


(512, 512)
Done subset_10_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101122_20240131T101151_052349_065480_4193_0_2048.tif


...

(512, 512)
Done subset_9_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101122_20240131T101151_052349_065480_4193_512_2048.tif


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


(512, 512)
Done subset_9_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101122_20240131T101151_052349_065480_4193_512_2496.tif


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


(512, 512)
Done subset_9_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240131T101122_20240131T101151_052349_065480_4193_512_512.tif
All shapefiles deleted successfully.
All shapefiles deleted successfully.
All shapefiles deleted successfully.
All shapefiles deleted successfully.
All shapefiles deleted successfully.


In [76]:
cleanup("./shapefiles/")

All shapefiles deleted successfully.
All shapefiles deleted successfully.
All shapefiles deleted successfully.
All shapefiles deleted successfully.
All shapefiles deleted successfully.
